In [1]:
#Note client and connection data was removed after running. This will not run without those credentials. 

import os
from binance.client import Client
import pandas as pd
import datetime, time
import numpy as np
from psycopg2 import connect
from decimal import Decimal
import mplfinance as mpf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

client = Client(<>, <>)
conn = connect(user=<>, password=<>, dbname=<>)
cur = conn.cursor()
symbol =  ["ADAUSDT","BTCUSDT","GALAUSDT","ETHUSDT","CRVUSDT","BNBUSDT","LTCUSDT","SHIBUSDT"
         ,"AVAXUSDT","SOLUSDT"]
interval="1d"


In [2]:
def get_latest_date(conn):
    cur.execute(""" Select "Open_T" FROM "Cryptodata_Day" order by "Open_T" desc limit 1 """)
    last_end_date = cur.fetchone()[0]
    return last_end_date
    conn.commit()
    
def prep_update(conn, ticker, last_end_date):
    query = """
    Delete From "Cryptodata_Day" 
    Where "Symbol" = '{}'
    And "Open_T" > '{}'
    """.format(ticker,last_end_date)
    cur.execute(query)
    conn.commit()
    
def single_inserts(conn, df):
    for i in df.index:
        cols  = ','.join(list(df.columns))
        vals  = [df.at[i,col] for col in list(df.columns)]
        query = """INSERT INTO "Cryptodata_Day"("Symbol", "Open_T", "Close_T", "Open"
        , "High", "Low", "Close", "Volume", "Interval") 
        VALUES('{}','{}','{}','{}','{}'
        ,'{}','{}'
        ,'{}','{}')""".format(vals[7], vals[0], vals[6]
                         ,vals[1],vals[2], vals[3]
                         , vals[4],vals[5],vals[8])
        cur.execute(query)
        conn.commit()   
        
def delete_data(conn):
    query = """Delete From "Cryptodata_Day" """
    cur.execute(query)
    conn.commit()

def get_data(conn):
    cur.execute(""" Select * FROM "Cryptodata_Day" Order by "Open_T" desc """)
    sql_data = cur.fetchall()
    return sql_data

# First load of selected cryptocurrencies

In [3]:
start="2021-02-16T00:00:00Z"
end=datetime.datetime.now().strftime("%Y/%d/%m")
for sym in symbol:
    try:
        klines = client.get_historical_klines(sym, interval, start_str=start, end_str=end)
        hist_df = pd.DataFrame(klines)
        hist_df.columns = ['Open_Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_Time', 'Quote_Asset_Volume', 
                            'Number_of_Trades', 'TB_Base_Volume', 'TB_Quote_Volume', 'Ignore']
        hist_df['Open_Time'] = pd.to_datetime(hist_df['Open_Time']/1000, unit='s')
        hist_df['Close_Time'] = pd.to_datetime(hist_df['Close_Time']/1000, unit='s')
        hist_df['Symbol'] = sym
        hist_df['Interval'] = interval
        hist_df.drop(columns=['Ignore'], inplace=True)
        numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote_Asset_Volume', 'TB_Base_Volume', 'TB_Quote_Volume']
        hist_df[numeric_columns] = hist_df[numeric_columns].apply(pd.to_numeric, axis=1)
        df_send = hist_df[['Open_Time','Open','High','Low','Close','Volume','Close_Time', 'Symbol','Interval']]
        df_send
        df=df_send
        single_inserts(conn, df)
    except:
        pass


# Update to today for selected cryptocurrencies

In [4]:
last_sql_time = get_latest_date(conn)
last_sql_time=last_sql_time.strftime("%Y-%m-%dT%H:%M:%S%z")
last_end_date = get_latest_date(conn)
start=last_sql_time
end=datetime.datetime.now().strftime("%Y/%d/%m")

# Deletes the lastest day's information in database
for ticker in symbol:
    prep_update(conn, ticker, last_end_date)

#Inserts data starting from the deleted day's data and then inserts todays most recent data
for sym in symbol:
    try:
        klines = client.get_historical_klines(sym, interval, start_str=last_sql_time, end_str=end)
        hist_df = pd.DataFrame(klines)
        hist_df.columns = ['Open_Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_Time', 'Quote_Asset_Volume', 
                            'Number_of_Trades', 'TB_Base_Volume', 'TB_Quote_Volume', 'Ignore']
        hist_df['Open_Time'] = pd.to_datetime(hist_df['Open_Time']/1000, unit='s')
        hist_df['Close_Time'] = pd.to_datetime(hist_df['Close_Time']/1000, unit='s')
        hist_df['Symbol'] = sym
        hist_df['Interval'] = interval
        hist_df.drop(columns=['Ignore'], inplace=True)
        numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote_Asset_Volume', 'TB_Base_Volume', 'TB_Quote_Volume']
        hist_df[numeric_columns] = hist_df[numeric_columns].apply(pd.to_numeric, axis=1)
        df_send = hist_df[['Open_Time','Open','High','Low','Close','Volume','Close_Time', 'Symbol','Interval']]
        df_send
        df=df_send
        single_inserts(conn, df)
    except:
        pass

# Results from database

In [5]:
connect = get_data(conn)
df_out = pd.DataFrame()
for i in connect:
    data = {
        'Symbol':i[0],
        'Open_T':i[1],
        'Close_T':(i[2]),
        'Open':(i[3]),
        'High':(i[4]),
        'Low':(i[5]),
        'Close':(i[6]),
        'Volume':(i[7]),
        'Interval':(i[8])   
    }
    dt0=pd.DataFrame(data, index=['Symbol'])
    df_out=pd.concat([df_out,dt0],ignore_index=True)
print(df_out.head(25))
print(df_out.tail(25))

      Symbol                     Open_T                           Close_T  \
0    SOLUSDT  2022-03-18 00:00:00-07:00  2022-03-18 23:59:59.999000-07:00   
1    LTCUSDT  2022-03-18 00:00:00-07:00  2022-03-18 23:59:59.999000-07:00   
2    CRVUSDT  2022-03-18 00:00:00-07:00  2022-03-18 23:59:59.999000-07:00   
3    BTCUSDT  2022-03-18 00:00:00-07:00  2022-03-18 23:59:59.999000-07:00   
4    LTCUSDT  2022-03-18 00:00:00-07:00  2022-03-18 23:59:59.999000-07:00   
5   SHIBUSDT  2022-03-18 00:00:00-07:00  2022-03-18 23:59:59.999000-07:00   
6    SOLUSDT  2022-03-18 00:00:00-07:00  2022-03-18 23:59:59.999000-07:00   
7    ETHUSDT  2022-03-18 00:00:00-07:00  2022-03-18 23:59:59.999000-07:00   
8   SHIBUSDT  2022-03-18 00:00:00-07:00  2022-03-18 23:59:59.999000-07:00   
9    BNBUSDT  2022-03-18 00:00:00-07:00  2022-03-18 23:59:59.999000-07:00   
10   CRVUSDT  2022-03-18 00:00:00-07:00  2022-03-18 23:59:59.999000-07:00   
11  GALAUSDT  2022-03-18 00:00:00-07:00  2022-03-18 23:59:59.999000-07:00   